In [1]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
hope.config.prefix = "hope"
import numba
import numpy as np
from util import perf_comp_data
from native_util import load
%load_ext Cython
%load_ext version_information
%version_information numpy, Cython, numba, hope

Software versions
Python 3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 6.1.0
OS Darwin 15.6.0 x86_64 i386 64bit
numpy 1.13.1
Cython 0.26
numba 0.34.0
hope 0.6.1
Tue Aug 29 17:15:40 2017 CEST

# fibonacci

In [2]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)
hope_fib = hope.jit(fib)
numba_fib = numba.jit(fib, nopython=False)

native_fib_mod = load("fib")
native_fib = native_fib_mod.run 

n=20
assert fib(20) == 6765
assert hope_fib(20) == 6765
assert numba_fib(20) == 6765
assert native_fib(20) == 6765

running build_ext



In [3]:
%%cython

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef int cython_fib(int n):
    if n<2:
        return n
    return cython_fib(n-1)+cython_fib(n-2)

assert cython_fib(20) == 6765


In [4]:
%timeit fib(20)
%timeit hope_fib(20)
%timeit numba_fib(20)
%timeit cython_fib(20)
%timeit native_fib(20)

3.11 ms ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
39.5 µs ± 707 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
3.07 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
39.5 µs ± 571 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
39.6 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
perf_comp_data(["fib", "hope_fib", "numba_fib", "cython_fib", "native_fib"],
               5*["n"])

function: native_fib          , av. time sec:   0.00003945, min. time sec:   0.00003941, relative:       1.0
function: cython_fib          , av. time sec:   0.00003969, min. time sec:   0.00003967, relative:       1.0
function: hope_fib            , av. time sec:   0.00004080, min. time sec:   0.00004075, relative:       1.0
function: numba_fib           , av. time sec:   0.00341285, min. time sec:   0.00320110, relative:      86.5
function: fib                 , av. time sec:   0.00344686, min. time sec:   0.00311820, relative:      87.4


# quicksort

In [6]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    if False: return a
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                tmp = a[i]
                a[i] = a[j]
                a[j] = tmp
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

hope_qsort_kernel = hope.jit(qsort_kernel)
numba_qsort_kernel = numba.jit(qsort_kernel)

native_qsort_kernel_mod = load("qsort_kernel")
native_qsort_kernel = native_qsort_kernel_mod.run


running build_ext
building 'qsort_kernel' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'



In [7]:
def numpy_qsort_kernel(a, lo, hi):
    np.sort(a)

In [8]:
%%cython

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cython_qsort_kernel(np.double_t * a, int lo, int hi):
    cdef int i = lo
    cdef int j = hi
    cdef double pivot = 0
    cdef double tmp = 0.0
    if False: return a
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                tmp = a[i]
                a[i] = a[j]
                a[j] = tmp
                i += 1
                j -= 1
        if lo < j:
            _cython_qsort_kernel(a, lo, j)
        lo = i
        j = hi

def cython_qsort_kernel(np.ndarray[np.double_t, ndim=1] a, int lo, int hi):
    _cython_qsort_kernel(<np.double_t*> a.data, lo, hi)
    return a

building '_cython_magic_c4e853c051d00b69b625b262d253da82' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
clang: /Users/uweschmitt/.ipython/cython/_cython_magic_c4e853c051d00b69b625b262d253da82.c
In file included from /Users/uweschmitt/.ipython/cython/_cython_magic_c4e853c051d00b69b625b262d253da82.c:495:
In file included from /Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /Users/uweschmitt/Projects/hope/venv3/

In [9]:
lst = np.random.random(5000)


In [10]:
psorted = qsort_kernel(lst.copy(), 0, len(lst)-1)
hsorted = hope_qsort_kernel(lst.copy(), 0, len(lst)-1)
#nsorted = numba_qsort_kernel(lst.copy(), 0, len(lst)-1)
csorted = cython_qsort_kernel(lst.copy(), 0, len(lst)-1)
nasorted = native_qsort_kernel(lst.copy(), 0, len(lst)-1)


In [11]:
assert np.all(psorted[:-1] <= psorted[1:])
#assert np.all(hope_qsort_kernel[:-1] <= hope_qsort_kernel[1:])
#assert np.all(numba_qsort_kernel[:-1] <= numba_qsort_kernel[1:])
#assert np.all(cython_qsort_kernel[:-1] <= cython_qsort_kernel[1:])

%timeit qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit hope_qsort_kernel(lst.copy(), 0, len(lst)-1)
#%timeit numba_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit cython_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit native_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit np.sort(lst.copy())

22.9 ms ± 785 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
349 µs ± 3.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
332 µs ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
313 µs ± 8.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
242 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
a = lst.copy()

lo = 0
hi = len(lst)-1

perf_comp_data(["hope_qsort_kernel", 
                "qsort_kernel", 
                #"numpy_qsort_kernel", 
                "cython_qsort_kernel", 
                "native_qsort_kernel"],
               5*["a, lo, hi"], rep=100, extra_setup="from __main__ import lst;a = lst.copy()")

function: native_qsort_kernel , av. time sec:   0.00029402, min. time sec:   0.00029235, relative:       1.0
function: cython_qsort_kernel , av. time sec:   0.00031540, min. time sec:   0.00031303, relative:       1.1
function: hope_qsort_kernel   , av. time sec:   0.00035815, min. time sec:   0.00033481, relative:       1.2
function: qsort_kernel        , av. time sec:   0.02226692, min. time sec:   0.02194059, relative:      75.7


# pi sum

In [13]:
def pisum():
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            sum += 1.0/(k*k)
    return sum

def pisum_opt():
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            f += 1.
            sum += 1.0/(f*f)
    return sum

hope_pisum = hope.jit(pisum)
hope_pisum_opt = hope.jit(pisum_opt)

numba_pisum = numba.jit(pisum, nopython=True)
numba_pisum_opt = numba.jit(pisum_opt, nopython=True)

native_pisum_mod = load("pisum")
native_pisum = native_pisum_mod.run

native_pisum_opt_mod = load("pisum_opt")
native_pisum_opt = native_pisum_opt_mod.run


assert abs(pisum()-1.644834071848065) < 1e-6
assert abs(hope_pisum()-1.644834071848065) < 1e-6
assert abs(hope_pisum_opt()-1.644834071848065) < 1e-6
assert abs(numba_pisum()-1.644834071848065) < 1e-6
assert abs(native_pisum()-1.644834071848065) < 1e-6
assert abs(native_pisum_opt()-1.644834071848065) < 1e-6

running build_ext
building 'pisum' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'

running build_ext
building 'pisum_opt' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/incl

AssertionError: 

In [15]:
%%cython

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.locals(f=float)
@cython.cdivision(True)
def cython_pisum():
    cdef double sum = 0.0
    cdef int j, k
    for j in range(1, 501):
        sum = 0.0
        for k in range(1, 10001):
            sum += 1.0/(k*k)
    return sum

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.locals(f=float)
@cython.cdivision(True)
def cython_pisum_opt():
    cdef double sum = 0.0
    cdef int j, k
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            f += 1.
            sum += 1.0/(f*f)
    return sum


assert abs(cython_pisum()-1.644834071848065) < 1e-6
assert abs(cython_pisum_opt()-1.644834071848065) < 1e-6

In [16]:
%timeit pisum()
%timeit pisum_opt()
%timeit hope_pisum()
%timeit hope_pisum_opt()
%timeit numba_pisum()
%timeit numba_pisum_opt()
%timeit cython_pisum()
%timeit cython_pisum_opt()
%timeit native_pisum()
%timeit native_pisum_opt()

740 ms ± 38.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
615 ms ± 8.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
39 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.9 ms ± 401 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.9 ms ± 547 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.8 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.8 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.8 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
23.3 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
perf_comp_data(["pisum", "pisum_opt", 
                "hope_pisum", "hope_pisum_opt", 
                "numba_pisum", "numba_pisum_opt", 
                "cython_pisum", 
                "cython_pisum_opt",
                "native_pisum", "native_pisum_opt",], 
                None, rep=100)

function: cython_pisum_opt    , av. time sec:   0.02031763, min. time sec:   0.02003632, relative:       1.0
function: cython_pisum        , av. time sec:   0.02036748, min. time sec:   0.02007828, relative:       1.0
function: hope_pisum_opt      , av. time sec:   0.02039336, min. time sec:   0.02009269, relative:       1.0
function: numba_pisum         , av. time sec:   0.02047833, min. time sec:   0.02018560, relative:       1.0
function: numba_pisum_opt     , av. time sec:   0.02050058, min. time sec:   0.02010900, relative:       1.0
function: native_pisum_opt    , av. time sec:   0.02051785, min. time sec:   0.02011683, relative:       1.0
function: native_pisum        , av. time sec:   0.02956523, min. time sec:   0.02909494, relative:       1.5
function: hope_pisum          , av. time sec:   0.03828400, min. time sec:   0.03774484, relative:       1.9
function: pisum_opt           , av. time sec:   0.62492522, min. time sec:   0.60418793, relative:      30.8
function: pisum    